# Get Patient Opinion Based On Given SED XLSX
----

# Import Libraries

In [ ]:
import requests
import pandas as pd
import time
import configparser

# Read Config File

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
input_file = config['DEFAULT']['Input-File']
output_file = config['DEFAULT']['Output-File']
sed_area = config['DEFAULT']['SED-Area']
auth_key = config['DEFAULT']['Auth-Key']

## This is the last data collection from PO

In [ ]:
sed_df = pd.read_excel(input_file, index_col=0)
sed_df = sed_df[sed_df['X'].notnull()]
sed_df.shape

In [ ]:
sed_df.info()

In [ ]:
grouped = sed_df.groupby('county').groups
grouped.keys()

In [ ]:
# nsw_seds = list(grouped['NSW'])
# nt_seds = list(grouped['NT'])
# qld_seds = list(grouped['QLD'])
# sa_seds = list(grouped['SA'])
# vic_seds = list(grouped['VIC'])
wa_seds = list(grouped[sed_area])

## Let's form the query

In [ ]:
op_df = pd.DataFrame()

## lets collect data per state

In [ ]:
start_date = '2016-04-26'
end_date = '2020-06-25'
take_param = '1000000'
start = time.time()
for sed in wa_seds:
    area_param = sed
    print(area_param)
    query='https://www.careopinion.org.au/api/v2/opinions?=frompopulation'+ area_param + '&publishedafter=' + start_date + '&publishedbefore=' + end_date + '&take=' + take_param
    response = requests.get(url=query,headers={'Authorization': 'SUBSCRIPTION_KEY {}'.format(auth_key)})
    if response.status_code == 200:
        print('Success!')
        response.encoding = 'utf-8'
        local_df = pd.DataFrame.from_dict(response.json())
        local_df['state'] = 'WA'
        local_df['nacs'] = area_param
        local_df['X'] = sed_df.at[area_param,'X']
        local_df['Y'] = sed_df.at[area_param,'Y']
        op_df = op_df.append(local_df)
    else:
        print('Failed!')
        
print("Used time {} seconds.".format(time.time()-start))

In [ ]:
op_df.shape

In [ ]:
op_df.head()

In [ ]:
op_df.tail()

# Export Output

In [ ]:
op_df.to_excel(output_file, index=False) 